In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm
from datetime import timedelta, datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import datetime

C:\Users\ACER\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [32]:
df = pd.read_csv('GlobalLandTemperaturesByCountry.csv',header=None,
                   names=['dt', 'AverageTemperature', 'AverageTemperatureUncertainty','Country'])
df[df['Country'] == 'Vietnam']
df2 = pd.DataFrame()
df2['Year'] = df[df['Country'] == 'Vietnam']['dt']
df2['Avg_Temp'] = df[df['Country'] == 'Vietnam']['AverageTemperature']
df2 = df2[:-1]

In [37]:
df2

,Avg_Temp
Year,
1825-01-01,19.539
1825-02-01,19.797
1825-03-01,22.176
1825-04-01,25.197
1825-05-01,26.235
...,...
2013-04-01,25.887
2013-05-01,27.443
2013-06-01,27.623


## 1. Facebook Prophet

In [33]:
def prophet(df):
    df2['Year'] = df2['Year'].apply(pd.to_datetime)
    df2.set_index('Year', inplace = True)
    df_train = df2.iloc[:1800]
    df_test = df2.iloc[1800:]
    
    from fbprophet import Prophet
    pp = Prophet()
    df_prophet = pd.DataFrame()
    df_prophet['ds'] = df_train.index.values
    df_prophet['y'] = df_train['Avg_Temp'].values
    pp.fit(df_prophet)
    future = pd.DataFrame(data = df_test.index.values, columns = ['ds'])
    forecast = pp.predict(future)
    
    print("RMSE:", np.sqrt(mean_squared_error(df_test, forecast['yhat'])))
    print("MAE:", mean_absolute_error(df_test, forecast['yhat']))
    print("R square score:", r2_score(df_test, forecast['yhat']))

In [34]:
prophet(df2)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\Users\ACER\Anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


RMSE: 0.756842268130477
MAE: 0.5680797875396175
R square score: 0.9294796206120167


In [83]:
len(df2['Avg_Temp'].values.reshape(1,2264).flatten())

2264

In [96]:
df2[df2['Avg_Temp'].isnull()]

,Avg_Temp
Year,
1826-01-01,NaN
1826-02-01,NaN
1826-03-01,NaN
1826-04-01,NaN
1826-05-01,NaN
...,...
1862-08-01,NaN
1862-09-01,NaN
1862-10-01,NaN


In [88]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer()
trans_data = imputer.fit_transform(df2['Avg_Temp'].values.reshape(1,2264))
trans_data.flatten()

array([19.539, 19.797, 22.176, ..., 27.623, 27.109, 27.026])

In [103]:
imputer.fit_transform(X = [[1,2,3,4,5,np.nan,np.nan,np.nan,9],[1,2,3,4,5,6,np.nan,7,9],[1,2,3,4,5,np.nan,np.nan,8,9]])

array([[1. , 2. , 3. , 4. , 5. , 6. , 7.5, 9. ],
       [1. , 2. , 3. , 4. , 5. , 6. , 7. , 9. ],
       [1. , 2. , 3. , 4. , 5. , 6. , 8. , 9. ]])